# Download Data

In [1]:
!wget -O snaphsot_serengeti_cropped_single_animals.zip "https://zenodo.org/records/11102888/files/snaphsot_serengeti_cropped_single_animals.zip?download=1"


--2024-11-19 20:44:40--  https://zenodo.org/records/11102888/files/snaphsot_serengeti_cropped_single_animals.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20480015768 (19G) [application/octet-stream]
Saving to: ‘snaphsot_serengeti_cropped_single_animals.zip’

snaphsot_serengeti_   0%[                    ]  18.00M  6.65MB/s               ^C


In [2]:
# Download train dataset
!wget -O train_dataset_crops_single_animal_template_captions_T1T7_ID.csv "https://zenodo.org/records/11102888/files/train_dataset_crops_single_animal_template_captions_T1T7_ID.csv?download=1"

# Download test dataset
!wget -O test_dataset_crops_single_animal_template_captions_T1T8T10.csv "https://zenodo.org/records/11102888/files/test_dataset_crops_single_animal_template_captions_T1T8T10.csv?download=1"

# Download validation dataset
!wget -O val_dataset_crops_single_animal_template_captions_T1T7_ID.csv "https://zenodo.org/records/11102888/files/val_dataset_crops_single_animal_template_captions_T1T7_ID.csv?download=1"


--2024-11-19 20:44:54--  https://zenodo.org/records/11102888/files/train_dataset_crops_single_animal_template_captions_T1T7_ID.csv?download=1
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 894463239 (853M) [text/plain]
Saving to: ‘train_dataset_crops_single_animal_template_captions_T1T7_ID.csv’

  train_dataset_cro   1%[                    ]  10.57M  5.18MB/s               ^C
--2024-11-19 20:44:58--  https://zenodo.org/records/11102888/files/test_dataset_crops_single_animal_template_captions_T1T8T10.csv?download=1
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 576150392 (549M) [text/plain]
Saving to: ‘test_dataset_crops_single_animal_template_caption

In [ ]:
!unzip snaphsot_serengeti_cropped_single_animals.zip


Streaming output truncated to the last 5000 lines.
  inflating: snaphsot_serengeti_cropped_single_animals/S4/R08/S4_R08_R1_IMAG0903_crop_0.jpg  
  inflating: snaphsot_serengeti_cropped_single_animals/S4/R08/S4_R08_R1_IMAG0902_crop_0.jpg  
  inflating: snaphsot_serengeti_cropped_single_animals/S4/R08/S4_R08_R1_IMAG0897_crop_0.jpg  
  inflating: snaphsot_serengeti_cropped_single_animals/S4/R08/S4_R08_R2_IMAG0365_crop_0.jpg  
  inflating: snaphsot_serengeti_cropped_single_animals/S4/R08/S4_R08_R2_IMAG0373_crop_0.jpg  
  inflating: snaphsot_serengeti_cropped_single_animals/S4/R08/S4_R08_R1_IMAG0900_crop_0.jpg  
  inflating: snaphsot_serengeti_cropped_single_animals/S4/R08/S4_R08_R2_IMAG0540_crop_0.jpg  
  inflating: snaphsot_serengeti_cropped_single_animals/S4/R08/S4_R08_R2_IMAG0350_crop_0.jpg  
  inflating: snaphsot_serengeti_cropped_single_animals/S4/R08/S4_R08_R2_IMAG0366_crop_0.jpg  
  inflating: snaphsot_serengeti_cropped_single_animals/S4/R08/S4_R08_R2_IMAG0515_crop_0.jpg  
  inflati

In [ ]:
!rm snaphsot_serengeti_cropped_single_animals.zip

# Organize data

In [3]:
import os
import pandas as pd
import shutil

In [4]:
!mkdir -p data
df_train=pd.read_csv('train_dataset_crops_single_animal_template_captions_T1T7_ID.csv')
df_val=pd.read_csv('val_dataset_crops_single_animal_template_captions_T1T7_ID.csv')
df_test=pd.read_csv('test_dataset_crops_single_animal_template_captions_T1T8T10.csv')


ParserError: Error tokenizing data. C error: EOF inside string starting at row 2793

In [5]:
info_folder={'train':df_train,'val':df_val,'test':df_test}
path='snaphsot_serengeti_cropped_single_animals'
for split_folder,info in info_folder.items():
  print(split_folder)
  for _,row in info.iterrows():
    species_id=str(row['species_id'])
    crop_path=row['crop_path']
    crop_path=os.path.join(path,crop_path)
    if os.path.exists(crop_path):
      img_name=crop_path.split('/')[-1]
      move_path=os.path.join('data',split_folder,species_id)
      if not os.path.exists(move_path):
        os.makedirs(move_path)
      move_path=os.path.join('data',split_folder,species_id,img_name)
      shutil.copy(crop_path,move_path)

NameError: name 'df_train' is not defined

In [6]:
!rm -r snaphsot_serengeti_cropped_single_animals


rm: cannot remove 'snaphsot_serengeti_cropped_single_animals': No such file or directory


# Training the model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score,classification_report

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Define data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

cuda


In [ ]:
# Load datasets
data_dir = 'data'
datasets = {x: datasets.ImageFolder(root=f'{data_dir}/{x}', transform=data_transforms[x])
            for x in ['train', 'val', 'test']}
dataloaders = {x: DataLoader(datasets[x], batch_size=32, shuffle=True, num_workers=4)
               for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(datasets[x]) for x in ['train', 'val', 'test']}
class_names = datasets['train'].classes
num_classes = len(class_names)

# Load pre-trained ResNet model and modify it for transfer learning
model = models.resnet18(pretrained=True)
# Replace the last fully connected layer with a new one (for our number of classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNe

In [ ]:
# Training function
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

# Train the model
model = train_model(model, criterion, optimizer, num_epochs=2)

Epoch 1/2
----------
train Loss: 0.4996 Acc: 0.8589
val Loss: 0.2843 Acc: 0.9196
Epoch 2/2
----------


# Results

In [ ]:

# Test the model and calculate additional metrics
def test_model(model):
    model.eval()
    correct = 0
    total = 0

    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Append labels and predictions for metric calculations
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    # Calculate accuracy
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

    # Calculate confusion matrix
    conf_matrix = confusion_matrix(all_labels, all_predictions)
    print("Confusion Matrix:")
    print(conf_matrix)

    # Calculate precision, recall, and F1 score
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')

    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    print(f'F1 Score: {f1:.2f}')
    #
    # Calculate F1-score and other metrics
    report = classification_report(all_labels, all_predictions, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    print("Classification Report:")
    print(report_df)

    # Create a DataFrame for the confusion matrix
    cm_df = pd.DataFrame(conf_matrix, index=templ, columns=templ)
    # Save both confusion matrix and classification report to a CSV file
    combined_df = pd.concat([cm_df, report_df], axis=1)
    combined_df.to_csv('classification_report.csv', index=True)


# Evaluate the model on the test set
test_model(model)
